# Exercise 24

From PM10 measurement, the output must contain the sensors having at least 2 measurements > 50.

---

In [1]:
# As usual, clear the destination folder to prevent overwriting
!hdfs dfs -rm -r /user/s315054/ex_results/res_out_Ex24

rm: `/user/s315054/ex_results/res_out_Ex24': No such file or directory


In [2]:
inPath = "/data/students/bigdata_internet/exercises/Ex24/data/sensors.txt"
outPath = "/user/s315054/ex_results/res_out_Ex24"

In [3]:
in_df = spark.read.load(inPath, format="csv", header=True, inferSchema=True)
in_df.show()

+--------+-------------------+----+
|sensorid|               date|pm10|
+--------+-------------------+----+
|      s1|2016-01-01 00:00:00|20.5|
|      s2|2016-01-01 00:00:00|30.1|
|      s1|2016-01-02 00:00:00|60.2|
|      s2|2016-01-02 00:00:00|20.4|
|      s1|2016-01-03 00:00:00|55.5|
|      s2|2016-01-03 00:00:00|52.5|
+--------+-------------------+----+



In [4]:
# Isolate rows with PM10 > 50
filt_df = in_df.filter("pm10 > 50")
filt_df.show()

+--------+-------------------+----+
|sensorid|               date|pm10|
+--------+-------------------+----+
|      s1|2016-01-02 00:00:00|60.2|
|      s1|2016-01-03 00:00:00|55.5|
|      s2|2016-01-03 00:00:00|52.5|
+--------+-------------------+----+



In [8]:
df_out = filt_df.select('sensorid', 'pm10').groupBy('sensorid').agg({"pm10":"count"}).filter('count(pm10) >= 2')
df_out.show()

+--------+-----------+
|sensorid|count(pm10)|
+--------+-----------+
|      s1|          2|
+--------+-----------+



In [9]:
# SQL query
in_df.createOrReplaceTempView('PM10')

In [12]:
df_out_2 = spark.sql("SELECT sensorid, count(pm10) \
        FROM PM10 \
        WHERE pm10>50 \
        GROUP BY sensorid \
        HAVING count(pm10)>=2")
df_out_2.show()

+--------+-----------+
|sensorid|count(pm10)|
+--------+-----------+
|      s1|          2|
+--------+-----------+



In [13]:
# Saving to HDFS
df_out.write.csv(outPath, header=False)